In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 1. Import required packages"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import fine as fn\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import matplotlib.gridspec as gridspec\n",
    "from matplotlib.colors import LinearSegmentedColormap\n",
    "import csv\n",
    "import numpy as np\n",
    "from matplotlib.gridspec import GridSpec\n",
    "import seaborn as sns\n",
    "from PIL import Image\n",
    "import matplotlib.image as mpimg\n",
    "from pathlib import Path\n",
    "import os\n",
    "import warnings\n",
    "from matplotlib.ticker import FuncFormatter\n",
    "\n",
    "cwd = Path.cwd()\n",
    "\n",
    "np.random.seed(\n",
    "    42\n",
    ")  # Sets a \"seed\" to produce the same random input data in each model run"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Einlesen der pd.Series aus inputs.ipynb"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "stromverbr = pd.read_pickle(\"stromverbrauch.pkl\")\n",
    "heizungswasserverb = pd.read_pickle(\"heizungswasser.pkl\")\n",
    "strompreis_cost = pd.read_pickle(\"strompreis_cost.pkl\")\n",
    "strompreis_revenue = pd.read_pickle(\"strompreis_revenue.pkl\")\n",
    "strompreis_cost_sinkt = pd.read_pickle(\"strompreis_cost_sinkt.pkl\")\n",
    "strompreis_revenue_sinkt = pd.read_pickle(\"strompreis_revenue_sinkt.pkl\")\n",
    "strompreis_cost_steigt = pd.read_pickle(\"strompreis_cost_steigt.pkl\")\n",
    "strompreis_revenue_steigt = pd.read_pickle(\"strompreis_revenue_steigt.pkl\")\n",
    "strompreis_cost_schwankt = pd.read_pickle(\"strompreis_cost_schwankt.pkl\")\n",
    "strompreis_revenue_schwankt = pd.read_pickle(\"strompreis_revenue_schwankt.pkl\")\n",
    "stromemissionen = pd.read_pickle(\"stromemissionen.pkl\")\n",
    "gaspreis = pd.read_pickle(\"gaspreis.pkl\")\n",
    "gaspreis_sensi = pd.read_pickle(\"gaspreisSensi.pkl\")\n",
    "pv = pd.read_pickle(\"pv.pkl\")\n",
    "cop = pd.read_pickle(\"cop.pkl\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "df_strom_pr_em_2023 = pd.read_pickle(\"StromPreisEmissionen.pkl\") # €/MWh\n",
    "df_strom_pr_em_2023_sensi = pd.read_pickle(\"StromPreisEmissionenSensi.pkl\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Für die Sensitivitätsanalyse der CO2-Intensität des Stroms"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "367.2723005136986"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "stromemissionen.mean()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 115,
   "metadata": {},
   "outputs": [],
   "source": [
    "stromemissionen=0.143*stromemissionen"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 116,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "52.519938973458906"
      ]
     },
     "execution_count": 116,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "stromemissionen.mean()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 2. Create an energy system model instance"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Input parameters\n",
    "locations = {\"FabrikAachen\"}\n",
    "commodityUnitDict = {\n",
    "    \"strom\": r\"kW_el\", # sinnvolle Maßeinheiten pro Stunde\n",
    "    \"netzstrom\": r\"kW_el\",\n",
    "    \"erdgas\": r\"kW_CH4,LHV\",\n",
    "    \"CO2\": r\"gCO2/h\",\n",
    "    \"heizungswasser\": r\"kW_th\",\n",
    "}\n",
    "commodities = {\"strom\", \"netzstrom\", \"erdgas\", \"CO2\", \"heizungswasser\"}\n",
    "numberOfTimeSteps, hoursPerTimeStep = 8760, 1 # Index geht von 0 bis 8759\n",
    "costUnit, lengthUnit = \"Euro\", \"m\" # 1e3  wären 1000\n",
    "startYear = 2023\n",
    "\n",
    "# Code\n",
    "esM = fn.EnergySystemModel(\n",
    "    locations=locations,\n",
    "    commodities=commodities,\n",
    "    numberOfTimeSteps=numberOfTimeSteps,\n",
    "    commodityUnitsDict=commodityUnitDict,\n",
    "    hoursPerTimeStep=hoursPerTimeStep,\n",
    "    costUnit=costUnit,\n",
    "    lengthUnit=lengthUnit,\n",
    "    verboseLogLevel=0,\n",
    "    startYear=startYear,\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{}"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "esM.componentNames"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 3. Add commodity sources to the energy system model"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3.1 Electricity source\n",
    "### PV (Brown Field)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "bei Basisszeanrio: capacityFix=2000\n",
    "\n",
    "bei SensAnly: capacityMax"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "esM.add(\n",
    "    fn.Source(\n",
    "        esM=esM,\n",
    "        name=\"PV\",\n",
    "        commodity=\"strom\",\n",
    "        hasCapacityVariable=True,\n",
    "        # capacityVariableDomain wird als continuous angenommen\n",
    "        operationRateFix=pv,\n",
    "        capacityFix=2000, # kW\n",
    "        #capacityMax=4000,\n",
    "        investPerCapacity=947.10, # €/kW\n",
    "        opexPerCapacity=947.10 * 0.01, # €/kW\n",
    "        interestRate=0.05,\n",
    "        economicLifetime=20, # a\n",
    "        # yearlyFullLoadHoursMin bzw. Max ist bei gegebener operationRateMax und capacity nicht wichtig\n",
    "        # commodityRevenue=0.1047 # €/kWh\n",
    "    )\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Netzstrom (Brownfield)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "für Sensitivitätsanalyse ändern"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [],
   "source": [
    "esM.add(\n",
    "    fn.Source(\n",
    "        esM=esM,\n",
    "        name=\"Stromkauf von Spotmarkt\",\n",
    "        commodity=\"netzstrom\",\n",
    "        hasCapacityVariable=False,\n",
    "        commodityCostTimeSeries=strompreis_cost, # €/kWh\n",
    "        commodityRevenueTimeSeries=strompreis_revenue # €/kWh\n",
    "    )\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3.2 Gas (Brownfield)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "bei Sensitivitätsanalyse commodityCost ändern"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [],
   "source": [
    "esM.add(\n",
    "    fn.Source(\n",
    "        esM=esM,\n",
    "        name=\"Erdgaskauf\",\n",
    "        commodity=\"erdgas\",\n",
    "        hasCapacityVariable=False,\n",
    "        commodityCostTimeSeries=gaspreis # €/kWh\n",
    "        #commodityCost=0.0336\n",
    "    )\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 4. Add conversion components to the energy system model"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4.1 Stromnetzanschluss (Brownfield)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [],
   "source": [
    "esM.add(\n",
    "    fn.Conversion(\n",
    "        esM=esM,\n",
    "        name=\"Netzanschluss\",\n",
    "        physicalUnit=r\"kW_el\",\n",
    "        commodityConversionFactors={\n",
    "            \"netzstrom\": -1,\n",
    "            \"strom\": 1,\n",
    "            \"CO2\": stromemissionen, # g/kWh\n",
    "        },\n",
    "        hasCapacityVariable=False,\n",
    "    )\n",
    ")"
   ]
  },